<a href="https://colab.research.google.com/github/HrishiDhondge/NVIDIA_workshop/blob/main/Nemo_agent_toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The NVIDIA NeMo Agent toolkit for AI Agents

### Getting started with Pre-requisites

1. Git Large File Storage (LFS)
2. uv
3. Nemo-Agent-toolkit GitHub repo

In [ ]:
# Download and install git-lfs
!wget https://github.com/git-lfs/git-lfs/releases/download/v3.7.0/git-lfs-linux-amd64-v3.7.0.tar.gz

# Extract the files
!tar -xvf git-lfs-linux-amd64-v3.7.0.tar.gz
!git-lfs-3.7.0/./install.sh # Install

In [ ]:
# Install uv with one of these two ways
# curl -LsSf https://astral.sh/uv/install.sh | sh
!pip install uv

### Clone the Repository and Update Submodules

In [ ]:
# Clone NeMo-Agent-Toolkit repository
!git clone https://github.com/NVIDIA/NeMo-Agent-Toolkit.git aiqtoolkit
%cd aiqtoolkit

In [ ]:
#Initialize, fetch, and update submodules
!git submodule update --init --recursive

# Fetch the data sets
!git lfs install
!git lfs fetch
!git lfs pull

### Create and Sync the Environment

In [ ]:
# Creating the environment
!uv venv --seed .venv --python 3.11 -c
!source .venv/bin/activate

# syncronizing the environment
!uv sync --all-groups --all-extras

In [ ]:
# Setting up some environment varibales here
import os
os.environ["UV_CONSTRAINT"] = ""
os.environ["UV_BUILD_CONSTRAINT"] = ""
os.environ["UV_PRERELEASE"] = "if-necessary-or-explicit"
os.environ["UV_SYSTEM_PYTHON"] = "false"
os.environ["aiq"] = "./.venv/bin/aiq"


### Install Individual Plugins from `packages/` directory

In [ ]:
# Installing individual plugins
!uv pip install -e '.[langchain]'
# !uv pip install -e '.[llama-index]'
# !uv pip install -e examples/getting_started/simple_web_query

# Dependencies for profiling
!uv pip install -e '.[profiling]'

# Check the installed version for toolkit library
!$aiq --version

### **Define the NVIDIA API key**

In [ ]:
# Define your NVIDIA API key here
os.environ["NVIDIA_API_KEY"] = "<your_api_key>"

## Wikipedia Search Workflow

In [ ]:
filename='wikipedia_workflow.yaml'
content = """
functions:
   # Add a tool to search wikipedia
   wikipedia_search:
      _type: wiki_search
      max_results: 2

llms:
   # Tell NeMo Agent toolkit which LLM to use for the agent
   nim_llm:
      _type: nim
      model_name: meta/llama-3.1-70b-instruct
      temperature: 0.0

workflow:
   # Use an agent that 'reasons' and 'acts'
   _type: react_agent
   # Give it access to our wikipedia search tool
   tool_names: [wikipedia_search]
   # Tell it which LLM to use
   llm_name: nim_llm
   # Make it verbose
   verbose: true
   # Retry up to 3 times
   parse_agent_response_max_retries: 3
"""
# Writing to the file
with open(filename, "w") as f:
    f.write(content)

In [ ]:
!$aiq run --config_file wikipedia_workflow.yaml --input "List five subspecies of Aardvarks"

## Web Search

In [ ]:
# Install the plugin
!uv pip install -e examples/getting_started/simple_web_query

In [ ]:
# Run the workflow
!$aiq run --config_file examples/getting_started/simple_web_query/configs/config.yml --input "What is LangSmith?"

- **Tip:** If you want to search any other website, you can change the web url in the yml file: `examples/getting_started/simple_web_query/configs/config.yml`